In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import time
import copy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [2]:
map_amsterdam = gpd.read_file("Sidewalk_width_crossings_smaller.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map_amsterdam['length']
objective2 = map_amsterdam['crossing']
objective3 = map_amsterdam['obstacle_free_width']

objectives = ('length', 'crossing')

In [3]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map_amsterdam, approach='primal')
nodes = G.nodes
edges = G.edges
print(G)

MultiGraph with 179 nodes and 230 edges


/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [ ]:
# print(nodes)

In [4]:
#Pick random ones or pick manually that make sense - to experiment
S = (122245.37633330293, 486126.8581684635)
# T = (122246.77932030056, 486223.5791244763)
T = (122253.09793657108, 486219.18429932056)

In [30]:
import single_vi_iter
lower_length = single_vi_iter.single_value_iter(G, T, 'length')
lower_length

{(122245.37633330293, 486126.8581684635): 157.33,
 (122254.86602688645, 486129.80052856216): 147.34,
 (122264.35426393585, 486132.74411788705): 137.35,
 (122273.84081337851, 486135.69101398275): 127.35999999999999,
 (122283.19823912054, 486138.9678026403): 117.36999999999999,
 (122284.1816391946, 486139.19731384714): 116.35999999999999,
 (122287.28805009159, 486137.97386694513): 119.70999999999998,
 (122291.40829399273, 486140.643590483): 124.64999999999998,
 (122289.53424561612, 486128.31132216856): 129.7,
 (122297.7854183222, 486139.47700177913): 131.21999999999997,
 (122301.24255780602, 486130.1701925009): 141.20999999999998,
 (122304.47424118801, 486120.76208520425): 151.2,
 (122256.54763350038, 486170.91398380254): 96.14,
 (122247.04588395767, 486167.91526857053): 106.13,
 (122237.50744934028, 486164.9669819529): 116.11999999999999,
 (122227.969152229, 486162.0189639581): 126.10999999999999,
 (122218.43104837791, 486159.0709819753): 136.1,
 (122289.24584929895, 486173.25169782597)

In [35]:
def pareto_dominates(a, b):
    """Check if the vector in b Pareto dominates vector a.

    Note: The original code has been modified to work for our minimization problem.

    Args:
        a (ndarray): A numpy array.
        b (ndarray): A numpy array.

    Returns:
        bool: Whether vector b dominates vector a.
    """
    a = np.array(a)
    b = np.array(b)
    return np.all(a <= b) and np.any(a < b)


def p_prune(candidates):
    """Create a Pareto coverage set from a set of candidate points.

    References:
        .. [1] Roijers, D. M., & Whiteson, S. (2017). Multi-objective decision making. 34, 129–129.
            https://doi.org/10.2200/S00765ED1V01Y201704AIM034

    Args:
        candidates (Set[Tuple]): A set of vectors.

    Returns:
        Set[Tuple]: A Pareto coverage set.
    """
    pcs = set()
    while candidates:
        vector = candidates.pop()

        for alternative in candidates:
            if pareto_dominates(alternative, vector):
                vector = alternative

        to_remove = set(vector)
        for alternative in candidates:
            if pareto_dominates(vector, alternative):
                to_remove.add(alternative)

        candidates -= to_remove
        pcs.add(vector)
    return pcs


def pvi(G, T, objectives, max_iter=100, epsilon=0.05):
    start = time.time()
    nd_vectors = [[{tuple(np.full(2, np.inf))} for _ in range(len(G.nodes))] for _ in range(len(G.nodes))] # Initialisation of nodes
    is_last = False

    path = []

    for n, current_node in enumerate(G.nodes):
        if current_node == T:  # We've reached the terminal state
            # G_copy = G.copy()
            # G_copy.remove_edge(T,n)
            nd_vectors[n] = [{(0, 0)} for _ in G.nodes]
    # print(nd_vectors)

    nd_vectors_update = copy.deepcopy(nd_vectors)
    for run in range(max_iter):  # We execute the algorithm for a number of iterations.
        # print(f'Value Iteration number: {run}')
        for n, current_node in enumerate(G.nodes):  # Loop over all states. Note: current_node is an object; n=number
            if current_node == T:
                continue

            for nk, neighbor in enumerate(G.nodes): #Note: neighbor is an object; k=number
                if neighbor not in G.neighbors(current_node):
                    continue
                # if neighbor == T:
                #     raise
                edge = G[current_node][neighbor]
                edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties

                cost = []
                for i in objectives:
                  cost.append(edge_list[0][i])

                cost = np.array(cost)
                results = set()

                for v_list in nd_vectors[nk]:
                    # if neighbor == T:
                        # print(v_list)
                    for value_vec in v_list:
                        results.add(tuple(cost+value_vec))
                        # if np.all((0, 0) == value_vec):
                            # print(value_vec)
                # for value_vec in nd_vectors[n][nk]:
                #     results.add(tuple(map(sum, zip(cost, value_vec))))
                #     print(results)
                # results = {tuple(cost + value_vec) for v_list in nd_vectors[nk] for value_vec in v_list}  # The set of candidate vectors.
                results = p_prune(results)
                # print(results)
                # print(f"n: {n}; nk:{nk}")
                nd_vectors_update[n][nk] = results

                if np.any(nd_vectors_update[n][nk] == {(210.9, 1.0)}):
                    path.append(current_node)
                    path.append(neighbor)
                    current_node = neighbor

        if is_last:
            break
        nd_vectors = copy.deepcopy(nd_vectors_update)  # Else perform a deep copy and go again.


    end = time.time()
    elapsed_seconds = (end - start)
    print("Seconds elapsed: " + str(elapsed_seconds))

    return path


# for each action in A:
# retireve R=cost
# Q is set of value vector
# s' is neighbor
# T is 1
#actions = neighbors

# def track_policy(G, t, objectives, pvi_vec):
#     for n, current_node in enumerate(G.nodes):  # Loop over all states. Note: current_node is an object; n=number
#         if current_node == T:
#             break
#
#         for nk, neighbor in enumerate(G.neighbors(current_node)):
#             edge = G[current_node][neighbor]
#             edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties
#             cost = []
#             for i in objectives:
#                 cost.append(edge_list[0][i])
#
#             cost = np.array(cost)
#             for val_vec in pvi_vec:
#                 print(val_vec)
#                 print(cost)
#                 if (val_vec + cost) == t:
#                     current_node = neighbor
#                     t = val_vec


# def track_policy(G, target, pvi_vec, tol=1e-3):
#         """Track a policy from its return vector.
#
#         Args:
#
#             G: Graph
#             tol (float, optional): The tolerance for the return vector. (Default value = 1e-3)
#         """
#         # target = np.array(vec)
#         terminated = False
#         truncated = False
#         total_rew = np.zeros(2)
#         paths = [[]]
#
#         while not (terminated or truncated):
#             closest_dist = np.inf
#             closest_action = 0
#             found_action = False
#             new_target = target
#
#             for n, current_node in enumerate(G.nodes):
#                 for nk, neighbor in enumerate(G.neighbors(current_node)):
#                     im_rew = np.zeros((n, nk, 2))[n, nk]
#                     print(f"im_rew:{im_rew}")
#                     non_dominated_set = pvi_vec[n][nk]
#                     print(f"non_dominated set:{non_dominated_set}")
#
#                     for q in non_dominated_set:
#                         q = np.array(q)
#                         print(f"q:{q}")
#                         dist = np.sum(np.abs(q + im_rew - t))
#                         print(f"distance:{dist}")
#                         if dist < closest_dist:
#                             closest_dist = dist
#                             closest_action = nk
#                             new_target = q
#
#                             if dist < tol:
#                                 found_action = True
#                                 break
#
#                     if found_action:
#                         break
#
#                 #closest action = edge you want to take
#                 #step = go to neighbor node
#                 # Append the action to the last path
#                 paths[-1].append(closest_action)
#
#                 if truncated:
#                     paths.append([])  # Start a new path
#
#                 n = closest_action
#                 reward = im_rew
#                 terminated = True
#                 truncated = True
#                 total_rew += reward
#                 target = new_target
#
#         return paths


pvi_result = pvi(G, T, objectives)
print(pvi_result)
# t = np.array([165.21, 2.])
# path_pvi = track_policy(G, t, objectives, pvi_result)
# print(path_pvi)

Seconds elapsed: 70.11304688453674
[(122381.26079255168, 486280.2605662771), (122380.44329276506, 486283.2723370011), (122380.44329276506, 486283.2723370011), (122381.26079255168, 486280.2605662771), (122381.26079255168, 486280.2605662771), (122380.44329276506, 486283.2723370011), (122380.44329276506, 486283.2723370011), (122381.26079255168, 486280.2605662771), (122380.44329276506, 486283.2723370011), (122381.26079255168, 486280.2605662771), (122381.26079255168, 486280.2605662771), (122380.44329276506, 486283.2723370011), (122380.44329276506, 486283.2723370011), (122381.26079255168, 486280.2605662771), (122380.44329276506, 486283.2723370011), (122381.26079255168, 486280.2605662771), (122381.26079255168, 486280.2605662771), (122380.44329276506, 486283.2723370011), (122380.44329276506, 486283.2723370011), (122381.26079255168, 486280.2605662771), (122380.44329276506, 486283.2723370011), (122381.26079255168, 486280.2605662771), (122381.26079255168, 486280.2605662771), (122380.44329276506, 

In [ ]:
# def track_policy():
#         """Track a policy from its return vector.
#
#         Args:
#             vec (array_like): The return vector to track.
#             G: Graph
#             tol (float, optional): The tolerance for the return vector. (Default value = 1e-3)
#         """
#         # target = np.array(vec)
#         # state, _ = env.reset()
#         terminated = False
#         truncated = False
#         total_rew = np.zeros(self.num_objectives)
#
#         while not (terminated or truncated):
#             # state = np.ravel_multi_index(state, self.env_shape)
#             closest_dist = np.inf
#             closest_action = 0
#             found_action = False
#             new_target = target
#
#             for n, current_node in enumerate(G.nodes):
#                 for nk, neighbor in enumerate(G.neighbors(current_node)):
#                     im_rew = self.avg_reward[n, nk]
#                     non_dominated_set = self.non_dominated[n][nk]
#
#                     for q in non_dominated_set:
#                         q = np.array(q)
#                         dist = np.sum(np.abs(q + im_rew - t))
#                         if dist < closest_dist:
#                             closest_dist = dist
#                             closest_action = nk
#                             new_target = q
#
#                             if dist < tol:
#                                 found_action = True
#                                 break
#
#                     if found_action:
#                         break
#
#                 #closest action = edge you want to take
#                 #step = go to neighbor node
#                 state, reward, terminated, truncated, _ = G.nodes(closest_action) #go to the node that it's at the end of the edge
#                 total_rew += reward
#                 target = new_target
#
#         return total_rew
